In [5]:
from dataclasses import dataclass, field
from typing import List, Any, Dict, Set
from collections import Counter
from abc import ABC
import pprint
import re
import os
pp = pprint.PrettyPrinter(indent=4)    

In [6]:
@dataclass
class Word:
    word_init: str
    word_mod: str
    lemma_basic: Counter = field(default_factory=Counter)
    lemma_adv: Counter = field(default_factory=Counter)
    pos_tags: Counter = field(default_factory=Counter)
    freq: int = field(default=1)
    


def join_words(word1: Word, word2: Word) -> Word:
    return Word(word1.word_init, word1.word_mod,
                word1.lemma_basic + word2.lemma_basic, 
                word1.lemma_adv + word2.lemma_adv,
                word1.pos_tags + word2.pos_tags,
                word1.freq + word2.freq
                )
    
@dataclass
class WordDict(ABC):
    dictionary: Dict[str, Word] = field(default_factory=dict)
    
    def add(self, word: Word):
        if word.word_mod in self.dictionary:
            self.dictionary[word.word_mod] = join_words(word, self.dictionary[word.word_mod])
        else:
            self.dictionary[word.word_mod] = word
    
    def save(self, path: str) -> None:
        pass
    
class CSVWordDict(WordDict):
    header = ["word" ,'lemma', 'pos_tag', 'freq']
    def save(self, path: str) -> None:
        with open(path,'w', encoding='UTF8') as f:
            writer = csv.writer(f)
            writer.writerow(self.header)
            for _, word in self.dictionary.items():
                writer.writerow([word.word_mod, word.lemma_basic.most_common(1)[0][0], word.pos_tags.most_common(1)[0][0], word.freq])
            
        

In [7]:
DOC_TAG = 'document'
TEXT_TAG = 'text'
DIV_TAG = 'div'
PARAGRAPH_TAG = 'p'


WORD_PATTERN = re.compile(r'(?P<word_forms>.+?)¦(?P<lemma1>.*?)¦?'
                               r'.*?¦(?P<lemma2>.*?)¦.*?¦(?P<pos_tags>.+?)¦\s*(?P<title>.+)', re.DOTALL)
START_TAG_PATTERN = re.compile(r'<(?P<tag>\w+)(?P<attributes>(\s*?\w+?=.+?)*)>\s+(?P<title>.+)', 	re.DOTALL)
END_TAG_PATTERN = re.compile(r'<(/?(?P<tag>\w+)/?)>\s+(?P<title>.+)', 	re.DOTALL)

@dataclass
class CorpusParser:
    dict_: WordDict = field(default_factory=CSVWordDict)
    tag_stack: List[str] = field(default_factory=list)
    
    def handle_starttag(self, match: re.Match) -> None:
        tag = match.groupdict()['tag']
        self.tag_stack.append(tag)  # add tag on top of the stack, marking start of the tag
                
    def handle_endtag(self, match: re.Match) -> None:
        tag = match.groupdict()['tag']
        try:
            self.tag_stack.pop()  ## remove tag from the top of the stack, marking end of the tag
        except Exception:
            print(tag, match.group())
            
    def handle_parts(self, parts: List[str]):
        # title = "".join(parts[-1].strip().split())
        word_forms = parts[0]
        word_forms = word_forms.split()
        init_form = " ".join(word_forms[:len(word_forms) // 2 ])
        ext_form = " ".join(word_forms[len(word_forms) // 2: ])
        pos_tags = Counter(parts[-2].split())
        lemma_basic = Counter([parts[1]])
        lemma_adv = Counter(parts[3].split())
        word = Word(init_form, ext_form, lemma_basic, lemma_adv, pos_tags)
        self.dict_.add(word)

    def handle_word(self, line: str):
        curr_tag = self.tag_stack[-1]
        if curr_tag == PARAGRAPH_TAG or curr_tag == DIV_TAG:
            parts = line.split('¦')
            self.handle_parts(parts)
        # parts = line.split('¦')
        # self.handle_parts(parts)
            
    def finish_dict(self) -> WordDict:
        return self.dict_
            
    def feed(self, line: str) -> None:
        open_tag_match = START_TAG_PATTERN.match(line)
        closed_tag_match = END_TAG_PATTERN.match(line)
        if open_tag_match is not None:
            self.handle_starttag(open_tag_match)
        elif closed_tag_match is not None:
            self.handle_endtag(closed_tag_match)
        else:
            self.handle_word(line)
            

In [10]:
CSV_FILES_PATh = r'csv/'
PART_FILE_ROOT = r'parts/'

parser = CorpusParser()


source = "grc-cwb"
# source = 'parts/part9.txt'
dest = os.path.join(CSV_FILES_PATh, f"adv_dict2.csv")
print(source)
with open(source, "r") as f:
    for number, line in enumerate(f):
        parser.feed(line)
    parser.finish_dict().save(dest)
    


grc-cwb
document </document>					lib.ge	იოსიფ ბროდსკი	განვავრცობ პლატონს		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

text </text>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

document </document>					lib.ge	ბესო ხვედელიძე	მინდვრის ყვავილები		

div </div>					lib.ge	ბესო ხვედელიძე	ჯადო		

div </div>					lib.ge	ბესო ხვედელიძე	ჯადო		

div </div>					lib.ge	ბესო ხვედელიძე	ჯადო		

text </tex